# 单向匹配

## 单向匹配 录入男性评分，女性评分，拒绝列表

In [11]:
import pandas as pd
import numpy as np
import time

# 读取女性评分
with open("Scores_for_Women.txt", "r") as f:
    w_to_m_data = f.readlines()[1:]

# 转化为矩阵 w_to_m_score_matrix[i][j] 为 女性i对男性j的评分
w_to_m_Scores = []    
for i in range(len(w_to_m_data)):
    w_to_m_Scores.append([float(ii) for ii in w_to_m_data[i].strip('\n').split(' ')[:-1]])
    
w_to_m_score_matrix = np.array(w_to_m_Scores)

# 读取女性评分
with open("Scores_for_Men.txt", "r") as f:
    m_to_w_data = f.readlines()[1:]

# 转化为矩阵 m_to_w_score_matrix[i][j] 为 男性i对女性j的评分    
m_to_w_Scores = []    
for i in range(len(m_to_w_data)):
    m_to_w_Scores.append([float(ii) for ii in m_to_w_data[i].strip('\n').split(' ')[:-1]])
    
m_to_w_score_matrix = np.array(m_to_w_Scores)


# 读取拒绝列表
with open("Reject_List.txt", "r") as f:
    reject_data = f.readlines()[1:]

# 转化为列表 
reject_list = []    
for i in range(len(reject_data)):
    reject_list.append([int(ii) for ii in reject_data[i].strip('\n').split(' ')])
    
# print('reject_list', reject_list)

W_dict = {} # 女性可接受词典
M_dict = {} # 男性可接受词典

for i in range(68):
    W_dict[i] = [i for i in range(68)]
    M_dict[i] = [i for i in range(68)]

for j in reject_list: # 按拒绝列表移除不接受的用户
    W_dict[j[0]].remove(j[1])
    M_dict[j[1]].remove(j[0])


# One Side Matching

## 在此规则中，以男性匹配为例，同一名女性用户最多可以被推荐X次，一名男性最多只能收获一名推荐名额，目标1.最大化权重 2.最大化最差用户的权重

In [3]:
m_to_w_score_matrix 

array([[13.65, 14.65, 13.65, ..., 12.65,  7.15, 13.4 ],
       [31.25, 32.9 , 32.25, ..., 28.75, 26.25, 24.75],
       [21.5 , 47.  , 39.75, ..., 28.75, 37.5 , 45.  ],
       ...,
       [22.5 , 37.5 , 52.25, ..., 28.75, 33.75, 32.25],
       [37.5 , 47.5 , 63.75, ..., 46.25, 46.25, 22.25],
       [23.65, 14.65, 13.65, ..., 22.65,  7.15, 10.9 ]])

In [4]:
w_to_m_score_matrix

array([[30.25, 36.25, 24.  , ..., 37.5 , 35.  , 22.75],
       [26.25, 32.9 , 45.4 , ..., 33.75, 43.75, 16.25],
       [25.25, 37.25, 38.5 , ..., 52.25, 45.  , 14.  ],
       ...,
       [ 2.65,  4.65,  7.15, ...,  7.15,  7.15,  2.65],
       [16.25, 28.75, 33.75, ..., 33.75, 33.75, 26.25],
       [20.  , 22.25, 21.25, ..., 19.75, 27.25, 25.  ]])

In [12]:
from mip import *

# 模型一： 最大化匹配总权重
def OneSideMatching1(score_matrix, X, sex_side = 'male'): 
    # X: 每名用户(默认女性)可被推荐的最大次数
    # score_matrix： 用户的分数矩阵，为男性推荐则input为 m_to_w_score_matrix，为女性推荐则input为 w_to_m_score_matrix
    # sex_side： 为哪种性别的用户推荐， male为男性 female为女性
    
    OSM1 = Model(sense=MAXIMIZE, solver_name=GRB) # 生成模型 调用Gurobi求解器
    
    # 创建男女全集合，根据数据不同需要变动
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))
    
    # 添加二元变量

    x = [[OSM1.add_var(var_type=BINARY) for j in J] for i in I] # Decision Variable
    
    # 设定目标方程
    
    OSM1.objective = xsum(score_matrix[i][j]*x[i][j] for i in I for j in W_dict[i])
    
    # 根据不同性别添加约束
    if sex_side == 'male':
        for i in I:
            OSM1 += xsum(x[i][j] for j in W_dict[i]) <= X 

        for j in J:
            OSM1 += xsum(x[i][j] for i in M_dict[j]) <= 1 
            
    if sex_side == 'female':
        for i in I:
            OSM1 += xsum(x[i][j] for j in W_dict[i]) <= 1 

        for j in J:
            OSM1 += xsum(x[i][j] for i in M_dict[j]) <= X

    # 计算运行时间， 求解        
    start = time.perf_counter()
    status = OSM1.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start
    
    # 根据不同性别进行解的收集
    solution = []
    
    if sex_side == 'male':
        for i in I:
            for j in J:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    if sex_side == 'female':
        for j in J:
            for i in I:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    return solution, OSM1.objective_value, runningtime # 匹配结果，模型最优值，运行时间

In [13]:
sol = OneSideMatching1(w_to_m_score_matrix, 4, sex_side = 'female') 
sol[0]

[[41, 2],
 [37, 5],
 [15, 8],
 [63, 8],
 [35, 9],
 [44, 11],
 [66, 11],
 [0, 12],
 [3, 13],
 [29, 13],
 [14, 14],
 [9, 15],
 [48, 15],
 [65, 15],
 [22, 16],
 [26, 16],
 [56, 16],
 [67, 16],
 [24, 17],
 [13, 20],
 [51, 21],
 [7, 22],
 [53, 22],
 [54, 23],
 [17, 24],
 [32, 24],
 [40, 24],
 [61, 24],
 [55, 25],
 [31, 26],
 [6, 27],
 [45, 27],
 [50, 27],
 [38, 28],
 [62, 28],
 [19, 32],
 [49, 33],
 [36, 36],
 [58, 36],
 [1, 37],
 [18, 37],
 [20, 38],
 [43, 38],
 [60, 38],
 [11, 41],
 [8, 45],
 [12, 45],
 [59, 47],
 [64, 49],
 [25, 50],
 [30, 50],
 [52, 50],
 [57, 50],
 [2, 52],
 [34, 52],
 [4, 53],
 [16, 53],
 [42, 53],
 [46, 53],
 [5, 55],
 [28, 60],
 [39, 60],
 [47, 60],
 [33, 61],
 [21, 63],
 [23, 64],
 [27, 64],
 [10, 66]]

In [7]:
from mip import *

# 最大化最小权重
def OneSideMatching2(score_matrix, X, sex_side = 'male'):
    # X: 每名用户(默认女性)可被推荐的最大次数
    # score_matrix： 用户的分数矩阵，为男性推荐则input为 m_to_w_score_matrix，为女性推荐则input为 w_to_m_score_matrix
    # sex_side： 为哪种性别的用户推荐， male为男性 female为女性
    
    
    OSM2 = Model(sense=MAXIMIZE, solver_name=GRB) # 生成模型 调用Gurobi求解器
    
    # 创建男女全集合，根据数据不同需要变动         
    num_female = 68
    num_male = 68

    I = set(range(num_female))
    J = set(range(num_male))

    # 添加决策变量
    x = [[OSM2.add_var(var_type=BINARY) for j in J] for i in I]
    z = OSM2.add_var()

    # 设定目标方程
    OSM2.objective = z

    # 根据不同性别添加约束
    if sex_side == 'male':
        for i in I:
            OSM2 += xsum(x[i][j] for j in W_dict[i]) <= X 

        for j in J:
            OSM2 += xsum(x[i][j] for i in M_dict[j]) <= 1 
            OSM2 += xsum(score_matrix[i][j]*x[i][j] for i in M_dict[j]) >= z 
            
    if sex_side == 'female':
        for i in I:
            OSM2 += xsum(x[i][j] for j in W_dict[i]) <= 1 
            OSM2 += xsum(score_matrix[i][j]*x[i][j] for j in W_dict[i]) >= z 
            
        for j in J:
            OSM2 += xsum(x[i][j] for i in M_dict[j]) <= X

    # 计算运行时间， 求解       
    start = time.perf_counter()
    status = OSM2.optimize(max_seconds=600)
    end = time.perf_counter()
    runningtime = end-start
    
    # 根据不同性别进行解的收集
    solution = []
    
    if sex_side == 'male':
        for i in I:
            for j in J:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    
    if sex_side == 'female':
        for j in J:
            for i in I:
                if x[i][j].x > 0.99:
                    solution.append([i,j])
    
    return solution, OSM2.objective_value, runningtime # 匹配结果，模型最优值，运行时间

In [15]:
sol2 = OneSideMatching2(m_to_w_score_matrix, 2, sex_side = 'male')
sol2[0]

[[1, 8],
 [2, 60],
 [4, 18],
 [5, 30],
 [5, 31],
 [6, 13],
 [6, 58],
 [7, 3],
 [7, 54],
 [8, 45],
 [9, 2],
 [9, 21],
 [11, 23],
 [12, 10],
 [12, 32],
 [13, 35],
 [13, 38],
 [14, 22],
 [15, 39],
 [15, 65],
 [16, 40],
 [16, 67],
 [17, 66],
 [18, 55],
 [18, 64],
 [20, 9],
 [20, 20],
 [24, 14],
 [24, 36],
 [25, 12],
 [26, 1],
 [26, 43],
 [27, 24],
 [28, 25],
 [31, 0],
 [31, 17],
 [33, 53],
 [35, 7],
 [35, 52],
 [37, 62],
 [38, 11],
 [40, 47],
 [41, 57],
 [42, 50],
 [43, 6],
 [43, 44],
 [45, 15],
 [45, 26],
 [49, 28],
 [49, 33],
 [51, 19],
 [51, 37],
 [52, 46],
 [52, 56],
 [53, 16],
 [53, 42],
 [54, 5],
 [55, 48],
 [56, 41],
 [57, 49],
 [57, 51],
 [59, 61],
 [59, 63],
 [61, 4],
 [61, 59],
 [62, 34],
 [64, 29],
 [66, 27]]